In [ ]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, BatchNormalization, Dropout, LeakyReLU
from keras.optimizers import Adam

from rl.agents.sarsa import SARSAAgent
from rl.policy import BoltzmannQPolicy, BoltzmannGumbelQPolicy
from rl.memory import EpisodeParameterMemory

ENV_NAME = 'MsPacman-ram-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
#model = Sequential()
#model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
#model.add(Dense(nb_actions))
#model.add(Activation('softmax'))

# Option 2: deep network
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
#model.add(Dense(512))
#model.add(LeakyReLU(alpha=0.3))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(nb_actions))
model.add(Activation('tanh'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
#memory = EpisodeParameterMemory(limit=1000, window_length=1)

#cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
#               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=1000000, policy = BoltzmannGumbelQPolicy())
cem.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=10000000, visualize=False, verbose=2)

# After training is done, we save the best weights.
cem.save_weights('sarsa_{}_params.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=10, visualize=True)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

   15412/10000000: episode: 22, duration: 0.976s, episode steps: 446, steps per second: 457, episode reward: 230.000, mean reward: 0.516 [0.000, 10.000], mean action: 3.395 [1.000, 7.000], mean observation: 105.659 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   16295/10000000: episode: 23, duration: 1.929s, episode steps: 883, steps per second: 458, episode reward: 290.000, mean reward: 0.328 [0.000, 50.000], mean action: 3.983 [1.000, 7.000], mean observation: 109.552 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   16781/10000000: episode: 24, duration: 1.074s, episode steps: 486, steps per second: 453, episode reward: 210.000, mean reward: 0.432 [0.000, 10.000], mean action: 3.504 [1.000, 7.000], mean observation: 109.154 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   17412/10000000: episode: 25, duration: 1.372s, episode steps: 631, steps per second: 460, episode reward: 210.000, mean reward: 0.333 [0.000, 10.000], mean acti

   37545/10000000: episode: 52, duration: 2.026s, episode steps: 929, steps per second: 458, episode reward: 420.000, mean reward: 0.452 [0.000, 10.000], mean action: 3.602 [1.000, 7.000], mean observation: 107.118 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   38405/10000000: episode: 53, duration: 1.883s, episode steps: 860, steps per second: 457, episode reward: 470.000, mean reward: 0.547 [0.000, 50.000], mean action: 3.910 [1.000, 7.000], mean observation: 108.347 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   39006/10000000: episode: 54, duration: 1.311s, episode steps: 601, steps per second: 458, episode reward: 230.000, mean reward: 0.383 [0.000, 10.000], mean action: 3.872 [1.000, 7.000], mean observation: 109.539 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   39386/10000000: episode: 55, duration: 0.832s, episode steps: 380, steps per second: 457, episode reward: 100.000, mean reward: 0.263 [0.000, 10.000], mean acti

   57024/10000000: episode: 82, duration: 1.821s, episode steps: 834, steps per second: 458, episode reward: 290.000, mean reward: 0.348 [0.000, 50.000], mean action: 4.203 [1.000, 7.000], mean observation: 110.243 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   57888/10000000: episode: 83, duration: 1.889s, episode steps: 864, steps per second: 457, episode reward: 550.000, mean reward: 0.637 [0.000, 200.000], mean action: 3.824 [1.000, 7.000], mean observation: 108.914 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   58582/10000000: episode: 84, duration: 1.509s, episode steps: 694, steps per second: 460, episode reward: 230.000, mean reward: 0.331 [0.000, 10.000], mean action: 3.911 [1.000, 7.000], mean observation: 108.476 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
   59559/10000000: episode: 85, duration: 2.140s, episode steps: 977, steps per second: 457, episode reward: 420.000, mean reward: 0.430 [0.000, 50.000], mean act

...

  487957/10000000: episode: 712, duration: 1.515s, episode steps: 689, steps per second: 455, episode reward: 350.000, mean reward: 0.508 [0.000, 10.000], mean action: 3.820 [1.000, 7.000], mean observation: 108.629 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  488794/10000000: episode: 713, duration: 1.837s, episode steps: 837, steps per second: 456, episode reward: 510.000, mean reward: 0.609 [0.000, 200.000], mean action: 4.143 [1.000, 7.000], mean observation: 109.703 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  489352/10000000: episode: 714, duration: 1.227s, episode steps: 558, steps per second: 455, episode reward: 190.000, mean reward: 0.341 [0.000, 10.000], mean action: 3.590 [1.000, 7.000], mean observation: 108.261 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  489954/10000000: episode: 715, duration: 1.317s, episode steps: 602, steps per second: 457, episode reward: 250.000, mean reward: 0.415 [0.000, 10.000], mean

  507836/10000000: episode: 742, duration: 1.031s, episode steps: 456, steps per second: 442, episode reward: 130.000, mean reward: 0.285 [0.000, 10.000], mean action: 3.743 [1.000, 7.000], mean observation: 108.468 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  508562/10000000: episode: 743, duration: 1.625s, episode steps: 726, steps per second: 447, episode reward: 280.000, mean reward: 0.386 [0.000, 50.000], mean action: 3.682 [1.000, 7.000], mean observation: 109.008 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  509131/10000000: episode: 744, duration: 1.254s, episode steps: 569, steps per second: 454, episode reward: 140.000, mean reward: 0.246 [0.000, 10.000], mean action: 3.794 [1.000, 7.000], mean observation: 109.275 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  510008/10000000: episode: 745, duration: 1.915s, episode steps: 877, steps per second: 458, episode reward: 520.000, mean reward: 0.593 [0.000, 200.000], mean

  527153/10000000: episode: 772, duration: 1.392s, episode steps: 635, steps per second: 456, episode reward: 250.000, mean reward: 0.394 [0.000, 10.000], mean action: 3.499 [1.000, 7.000], mean observation: 105.217 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  527783/10000000: episode: 773, duration: 1.382s, episode steps: 630, steps per second: 456, episode reward: 220.000, mean reward: 0.349 [0.000, 10.000], mean action: 3.816 [1.000, 7.000], mean observation: 109.506 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  528453/10000000: episode: 774, duration: 1.471s, episode steps: 670, steps per second: 456, episode reward: 480.000, mean reward: 0.716 [0.000, 200.000], mean action: 3.858 [1.000, 7.000], mean observation: 110.193 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  529323/10000000: episode: 775, duration: 1.903s, episode steps: 870, steps per second: 457, episode reward: 490.000, mean reward: 0.563 [0.000, 200.000], mea

  547575/10000000: episode: 802, duration: 1.397s, episode steps: 640, steps per second: 458, episode reward: 190.000, mean reward: 0.297 [0.000, 10.000], mean action: 3.372 [1.000, 7.000], mean observation: 108.391 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  548241/10000000: episode: 803, duration: 1.453s, episode steps: 666, steps per second: 458, episode reward: 270.000, mean reward: 0.405 [0.000, 50.000], mean action: 4.147 [1.000, 7.000], mean observation: 109.226 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  548781/10000000: episode: 804, duration: 1.176s, episode steps: 540, steps per second: 459, episode reward: 270.000, mean reward: 0.500 [0.000, 10.000], mean action: 4.044 [1.000, 7.000], mean observation: 108.856 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  549653/10000000: episode: 805, duration: 1.916s, episode steps: 872, steps per second: 455, episode reward: 340.000, mean reward: 0.390 [0.000, 50.000], mean 

  569287/10000000: episode: 832, duration: 2.582s, episode steps: 910, steps per second: 352, episode reward: 600.000, mean reward: 0.659 [0.000, 200.000], mean action: 4.010 [0.000, 7.000], mean observation: 108.842 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  569765/10000000: episode: 833, duration: 1.343s, episode steps: 478, steps per second: 356, episode reward: 120.000, mean reward: 0.251 [0.000, 10.000], mean action: 3.996 [1.000, 7.000], mean observation: 109.309 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  570270/10000000: episode: 834, duration: 1.432s, episode steps: 505, steps per second: 353, episode reward: 90.000, mean reward: 0.178 [0.000, 10.000], mean action: 3.814 [1.000, 7.000], mean observation: 110.230 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --
  570871/10000000: episode: 835, duration: 1.698s, episode steps: 601, steps per second: 354, episode reward: 190.000, mean reward: 0.316 [0.000, 10.000], mean 